# Modelling Algorithms

In [4]:
# Import your libraries:


import os

%matplotlib inline

import numpy as np
import pandas as pd
from sklearn import metrics
import re

import sys
sys.path.append('../')

import re
from math import sqrt
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler , StandardScaler
from sklearn.ensemble import RandomForestRegressor,BaggingRegressor,GradientBoostingRegressor,AdaBoostRegressor
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from xgboost import XGBRegressor
import xgboost as xgb
from xgboost import cv
from sklearn.metrics import mean_squared_log_error,mean_squared_error, r2_score, mean_absolute_error 


import seaborn as sns

import matplotlib as mpl
import matplotlib.pyplot as plt

import warnings

warnings.filterwarnings('ignore')




# https://www.kaggle.com/c/diamonds-datamad0921

In [5]:
diamonds_df_train = pd.read_csv("../data/train_clean.csv")

In [6]:
diamonds_df_test = pd.read_csv("../data/test_clean.csv")

In [7]:
diamonds_df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40455 entries, 0 to 40454
Data columns (total 23 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   carat          40455 non-null  float64
 1   x              40455 non-null  float64
 2   y              40455 non-null  float64
 3   z              40455 non-null  float64
 4   price          40455 non-null  float64
 5   volume         40455 non-null  float64
 6   cut_Good       40455 non-null  int64  
 7   cut_Ideal      40455 non-null  int64  
 8   cut_Premium    40455 non-null  int64  
 9   cut_Very Good  40455 non-null  int64  
 10  color_E        40455 non-null  int64  
 11  color_F        40455 non-null  int64  
 12  color_G        40455 non-null  int64  
 13  color_H        40455 non-null  int64  
 14  color_I        40455 non-null  int64  
 15  color_J        40455 non-null  int64  
 16  clarity_IF     40455 non-null  int64  
 17  clarity_SI1    40455 non-null  int64  
 18  clarit

In [ ]:
# diamonds_df_test.info()

In [8]:

# Collect all RMSE Scores.

RMSE_Scores = []
models = ['Linear Regression' , 'Decision Tree','RandomForest Regression','XGBoost. GridSearching Bayesian','KNeighbours Regression','Suppot Vector Machine']

## 1. Linear Regression

In [9]:
X = diamonds_df_train.drop('price', axis=1)

In [10]:
X.shape

(40455, 22)

In [11]:
y = diamonds_df_train.price

#### Apply Feature Scaling although it's not neccessary in this case. But it surely helps.

In [12]:
sc = MinMaxScaler()

In [13]:
# usamos 20% test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [14]:
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [15]:
lr = LinearRegression()
lr.fit(X_train , y_train)
accuracies = cross_val_score(estimator = lr, X = X_train, y = y_train, cv = 5,verbose = 1)
y_pred = lr.predict(X_test)
print('')
print('####### Linear Regression #######')
print('Score : %.4f' % lr.score(X_test, y_test))
print(accuracies)

mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
rmse_lr = mean_squared_error(y_test, y_pred)**0.5
r2 = r2_score(y_test, y_pred)

print('')
print('MSE    : %0.2f ' % mse)
print('MAE    : %0.2f ' % mae)
print('RMSE   : %0.2f ' % rmse_lr)
print('R2     : %0.2f ' % r2)


RMSE_Scores.append(rmse_lr)


####### Linear Regression #######
Score : 0.7872
[0.96314691 0.98343924 0.98361664 0.98348665 0.98216703]

MSE    : 0.22 
MAE    : 0.11 
RMSE   : 0.47 
R2     : 0.79 


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    0.1s finished


## 2. Decission Tree

In [16]:
clf_dt = DecisionTreeRegressor()
clf_dt.fit(X_train , y_train)
accuracies = cross_val_score(estimator = clf_dt, X = X_train, y = y_train, cv = 5,verbose = 1)
y_pred = clf_dt.predict(X_test)
print('')
print('###### Decision Tree ######')
print('Score : %.4f' % clf_dt.score(X_test, y_test))
print(accuracies)

mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
rmse_dt = mean_squared_error(y_test, y_pred)**0.5
r2 = r2_score(y_test, y_pred)

print('')
print('MSE    : %0.2f ' % mse)
print('MAE    : %0.2f ' % mae)
print('RMSE   : %0.2f ' % rmse_dt)
print('R2     : %0.2f ' % r2)


RMSE_Scores.append(rmse_dt)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.



###### Decision Tree ######
Score : 0.9804
[0.9780656  0.97681831 0.98138271 0.97806781 0.97872551]

MSE    : 0.02 
MAE    : 0.09 
RMSE   : 0.14 
R2     : 0.98 


[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    0.9s finished


### Tuning Parameters

In [17]:
g_cv = GridSearchCV(DecisionTreeRegressor(),
                    param_grid={
                        "max_depth": [2, 3, 7],
                        "min_samples_split": [30, 50, 100],

                        
                    },
                    scoring = "neg_mean_squared_error", 
                    refit = "neg_mean_squared_error",
                    cv=5, 
                    return_train_score = True)
#                     verbose = 2

In [18]:
g_cv.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=DecisionTreeRegressor(),
             param_grid={'max_depth': [2, 3, 7],
                         'min_samples_split': [30, 50, 100]},
             refit='neg_mean_squared_error', return_train_score=True,
             scoring='neg_mean_squared_error')

In [19]:
best_tree = g_cv.best_estimator_

In [20]:
rmse_train = round(sqrt(mean_squared_error(best_tree.predict(X_train), y_train)),5)
print('RMSE : %0.2f ' % rmse_train)

RMSE : 0.19 


In [21]:
rmse_test = round(sqrt(mean_squared_error(best_tree.predict(X_test), y_test)),4)
print('RMSE : %0.2f ' % rmse_test)

RMSE : 0.20 


## 3. RandomForest Regression

In [22]:
clf_rf = RandomForestRegressor()
clf_rf.fit(X_train , y_train)
accuracies = cross_val_score(estimator = clf_rf, X = X_train, y = y_train, cv = 5,verbose = 1)
y_pred = clf_rf.predict(X_test)
print('')
print('###### Random Forest ######')
print('Score : %.4f' % clf_rf.score(X_test, y_test))
print(accuracies)

mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
rmse_rf = mean_squared_error(y_test, y_pred)**0.5
r2 = r2_score(y_test, y_pred)

print('')
print('MSE    : %0.2f ' % mse)
print('MAE    : %0.2f ' % mae)
print('RMSE   : %0.2f ' % rmse_rf)
print('R2     : %0.2f ' % r2)

RMSE_Scores.append(rmse_rf)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:   57.6s finished



###### Random Forest ######
Score : 0.9894
[0.98825375 0.98703862 0.98893906 0.98822774 0.98772586]

MSE    : 0.01 
MAE    : 0.07 
RMSE   : 0.11 
R2     : 0.99 


### Tunning Parameters

In [23]:
no_of_test=[100]
params_dict={'n_estimators':no_of_test,'n_jobs':[-1],'max_features':["auto",'sqrt','log2']}
clf_rf=GridSearchCV(estimator = RandomForestRegressor(),param_grid=params_dict, scoring='neg_root_mean_squared_error')
clf_rf.fit(X_train,y_train)
pred = clf_rf.predict(X_test)
r2 = r2_score(y_test, pred)
rmse = mean_squared_error(y_test, pred)**0.5
print('R2     : %0.2f ' % r2)
print('RMSE     : %0.2f ' % rmse)


R2     : 0.99 
RMSE     : 0.11 


## 4. XGBoost. GridSearching Bayesian

In [24]:
from hyperopt import fmin, hp, tpe, Trials, space_eval, STATUS_OK
from hyperopt.pyll import scope as ho_scope
from hyperopt.pyll.stochastic import sample as ho_sample

In [25]:
xgbr = xgb.XGBRegressor()

In [26]:
space = {
    
    'n_estimators': hp.quniform('n_estimators', 10, 1000, 25), # uniforme discreta de 10 a 1000 en saltos de 25
    
    'learning_rate': hp.uniform('learning_rate', 0.0001, 1.0), # uniforme continua entre 0.0001 y 1
    
    'gamma': hp.uniform('x_gamma', 0.1, 0.5)
}

In [27]:
def objetivo(space):
    
    modelo = xgb.XGBRegressor(n_estimators=int(space['n_estimators']),
                            learning_rate=space['learning_rate'],
                            gamma = space['gamma'],
                            objective='reg:squarederror'
                           )
    
    eval_set = [(X_train, y_train), (X_test, y_test)]
    
    modelo.fit(X_train, y_train, eval_set = eval_set, eval_metric = 'rmse', verbose=False)
    
    y_pred = modelo.predict(X_test)
    
    rmse_boost = mean_squared_error(y_test, y_pred)**0.5
    
    return {'loss': rmse_boost, 'status': STATUS_OK}

In [28]:
mejor = fmin(fn = objetivo, space = space, algo = tpe.suggest, max_evals=10, trials=Trials())

mejor

100%|█████████| 10/10 [01:12<00:00,  7.25s/trial, best loss: 0.1036714007237022]


{'learning_rate': 0.4280480897702921,
 'n_estimators': 300.0,
 'x_gamma': 0.1275444402775237}

In [29]:
modelo=xgb.XGBRegressor(n_estimators=int(mejor['n_estimators']), 
                       learning_rate=mejor['learning_rate'], 
                       gamma=mejor['x_gamma'],
                       objective='reg:squarederror')

In [30]:
modelo.fit(X_train, y_train)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0.1275444402775237,
             gpu_id=-1, importance_type='gain', interaction_constraints='',
             learning_rate=0.4280480897702921, max_delta_step=0, max_depth=6,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=300, n_jobs=8, num_parallel_tree=1, random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
             tree_method='exact', validate_parameters=1, verbosity=None)

In [31]:
y_pred=modelo.predict(X_test)

rmse_boost = round((mean_squared_error(y_test, y_pred)**0.5),3)
print('RMSE     : %0.2f ' % rmse_boost)

RMSE_Scores.append(rmse_boost)

RMSE     : 0.10 


## 5. KNeighbours Regression

In [32]:
clf_knn = KNeighborsRegressor()
clf_knn.fit(X_train , y_train)
accuracies = cross_val_score(estimator = clf_knn, X = X_train, y = y_train, cv = 5,verbose = 1)
clf_knn.fit(X_train,y_train)
y_pred = clf_knn.predict(X_test)
print('')
print('###### KNeighbours Regression ######')
print('Score : %.4f' % clf_knn.score(X_test, y_test))
print(accuracies)

mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
rmse_kn = mean_squared_error(y_test, y_pred)**0.5
r2 = r2_score(y_test, y_pred)

print('')
print('MSE    : %0.2f ' % mse)
print('MAE    : %0.2f ' % mae)
print('RMSE   : %0.2f ' % rmse)
print('R2     : %0.2f ' % r2)

RMSE_Scores.append(rmse_kn)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:   18.9s finished



###### KNeighbours Regression ######
Score : 0.9800
[0.97736687 0.97803116 0.97788779 0.9775341  0.97867565]

MSE    : 0.02 
MAE    : 0.09 
RMSE   : 0.11 
R2     : 0.98 


In [33]:
n_neighbors=[]
for i in range (0,50,5):
    if(i!=0):
        n_neighbors.append(i)
params_dict={'n_neighbors':n_neighbors,'n_jobs':[-1]}
clf_knn=GridSearchCV(estimator=KNeighborsRegressor(),param_grid=params_dict,scoring='neg_root_mean_squared_error')
clf_knn.fit(X_train,y_train)
pred=clf_knn.predict(X_test)
rmse_kn2 = mean_squared_error(y_test, pred)**0.5
print('RMSE     : %0.2f ' % rmse_kn2)


RMSE     : 0.14 


## 6. Support Vector Machine

In [34]:
from sklearn.svm import SVR
regressor = SVR(kernel='rbf')
regressor.fit(X_train,y_train)
y_pred = regressor.predict(X_test)

In [35]:
rmse_svr = mean_squared_error(y_test, y_pred)**0.5
print('RMSE     : %0.2f ' % rmse_svr)
RMSE_Scores.append(rmse_svr)

RMSE     : 0.12 


In [36]:
compare = pd.DataFrame({'Algorithms' : models , 'RMSE_Scores' : RMSE_Scores})
compare.sort_values(by = 'RMSE_Scores' ,ascending=True)

,Algorithms,RMSE_Scores
3,XGBoost. GridSearching Bayesian,0.104000
2,RandomForest Regression,0.105328
5,Suppot Vector Machine,0.122656
1,Decision Tree,0.143383
4,KNeighbours Regression,0.144794
0,Linear Regression,0.471991


In [ ]:
# X_test2 = diamonds_df_test

In [ ]:
# X_test2 = sc.transform(X_test2)

In [ ]:
# y_pred = lr.predict(X_test2)

In [ ]:
# diamonds_df_test['price'] = y_pred

In [ ]:
# diamonds_df_test.head()

In [ ]:
# diamonds_df_test = diamonds_df_test.drop(["carat", "depth", "table", "x", "y", "z", "cut_Premium", "cut_Good","cut_Ideal", "cut_Very Good","color_E","color_F","color_G","color_H", "color_I", "color_J", "clarity_SI2", "clarity_IF", "clarity_SI1", "clarity_VS1", "clarity_VS2", "clarity_VVS1", "clarity_VVS2"], axis=1)

In [ ]:
# diamonds_df_test.head()

In [ ]:
# diamonds_df_test["id"] = diamonds_df_test.index

In [ ]:
# diamonds_df_test = diamonds_df_test.reindex(columns=['id','price'])

In [ ]:
# diamonds_df_test.shape

In [ ]:
# diamonds_df_test.head()

In [ ]:
# diamonds_df_test.to_csv("train.csv", index = False)